In [5]:
%pip install -q Faker==25.8.0

Note: you may need to restart the kernel to use updated packages.


In [22]:

from pathlib import Path
import numpy as np
import pandas as pd

ROOT = Path.cwd().resolve()
if ROOT.name == 'Parte01':
    ROOT = ROOT.parent

DATA_DIR = ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
REPORTS_FIGURES_DIR = ROOT / 'reports' / 'figures'
SQL_DIR = ROOT / 'sql'
PRESENTATION_DIR = ROOT / 'presentation'

for d in (RAW_DIR, PROCESSED_DIR, REPORTS_FIGURES_DIR, SQL_DIR, PRESENTATION_DIR):
    d.mkdir(parents=True, exist_ok=True)

SEED = 42
rng = np.random.default_rng(SEED)
np.random.seed(SEED)

print('Estrutura criada em:', ROOT)
print('Vers�es -> pandas:', pd.__version__, '| numpy:', np.__version__)


Estrutura criada em: C:\DEV\Teste_Analytics_CristovamPaulo
Vers�es -> pandas: 2.3.2 | numpy: 2.2.6


In [23]:
from datetime import datetime, timedelta
from faker import Faker
import numpy as np
import pandas as pd

EXTENDER_ATE_2024_06 = False
N_LINHAS = 300
START_2023 = "2023-01-01"
END_2023 = "2023-12-31"
END_DATE = "2024-06-30" if EXTENDER_ATE_2024_06 else END_2023

CATEGORIAS = {
    "Eletrônicos": ["Fone Pro", "Mouse Óptico", "Teclado Mecânico", "Smartwatch", "Caixa Bluetooth"],
    "Casa & Cozinha": ["Liquidificador Turbo", "Panela Antiaderente", "Aspirador 900", "Jogo de Facas", "Cafeteira X"],
    "Esporte": ["Bola Oficial", "Tênis Running", "Garrafa Térmica", "Mochila Fitness", "Camiseta Dry"],
    "Moda": ["Camiseta Básica", "Calça Jeans", "Jaqueta Leve", "Tênis Casual", "Boné Snapback"],
    "Brinquedos": ["Quebra-cabeça 500pç", "Carrinho RC", "Boneca Classic", "Blocos Criativos", "Jogo de Tabuleiro"],
}

PRECO_PARAMS = {
    "Eletrônicos":  {"mu": np.log(300), "sigma": 0.6, "minv": 50},
    "Casa & Cozinha":{"mu": np.log(150), "sigma": 0.5, "minv": 30},
    "Esporte":      {"mu": np.log(120), "sigma": 0.5, "minv": 20},
    "Moda":         {"mu": np.log(90),  "sigma": 0.5, "minv": 20},
    "Brinquedos":   {"mu": np.log(80),  "sigma": 0.6, "minv": 15},
}

SAZONALIDADE = {
    1: 0.90, 2: 0.95, 3: 1.00, 4: 1.00, 5: 1.05, 6: 1.00,
    7: 1.10, 8: 1.05, 9: 1.00, 10: 1.10, 11: 1.20, 12: 1.30,
}

def gerar_datas(n, start_date: str, end_date: str, rng: np.random.Generator):
    start = datetime.fromisoformat(start_date)
    end = datetime.fromisoformat(end_date)
    delta_dias = (end - start).days + 1
    offs = rng.integers(0, delta_dias, size=n)
    return np.array([start + timedelta(days=int(o)) for o in offs])

def preco_por_categoria(cat: str, rng: np.random.Generator) -> float:
    p = PRECO_PARAMS[cat]
    val = rng.lognormal(mean=p["mu"], sigma=p["sigma"])
    val = max(val, p["minv"])
    return round(float(val), 2)

rng = np.random.default_rng(SEED)
fake = Faker("pt_BR")

n = max(N_LINHAS, 50)

categorias = rng.choice(list(CATEGORIAS.keys()), size=n)
produtos = np. array([rng.choice(CATEGORIAS[c]) for c in categorias])

datas = gerar_datas(n, START_2023, END_DATE, rng)

lambda_base = 3.0
fatores = np.array([SAZONALIDADE.get(d.month, 1.0) for d in datas])
quantidade = rng.poisson(lam=lambda_base * fatores, size=n) + 1

precos = np.array([preco_por_categoria(cat, rng) for cat in categorias])

ids = np.arange(1, n + 1)

df = pd.DataFrame({
    "ID": ids,
    "Data": pd.to_datetime(datas).strftime("%Y-%m-%d"),
    "Produto": produtos,
    "Categoria": categorias,
    "Quantidade": quantidade,
    "Preco": precos,
})


raw_path = ROOT / "data/raw/vendas_2023.csv"
raw_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(raw_path, index=False, encoding="utf-8")

print(f"CSV bruto salvo em: {raw_path}")
print("Linhas totais (inclui duplicatas intencionais):", len(df))
df.head()


CSV bruto salvo em: C:\DEV\Teste_Analytics_CristovamPaulo\data\raw\vendas_2023.csv
Linhas totais (inclui duplicatas intencionais): 300


,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-08-13,Mouse Óptico,Eletrônicos,1,275.60
1,2,2023-10-12,Boné Snapback,Moda,4,139.26
2,3,2023-09-27,Camiseta Básica,Moda,5,68.88
3,4,2023-02-19,Camiseta Dry,Esporte,4,54.67
4,5,2023-10-29,Garrafa Térmica,Esporte,5,60.13


In [24]:
raw_path = ROOT / "data/raw/vendas_2023.csv"
df = pd.read_csv(raw_path)

In [25]:
n = len(df)

if "rgn" not in globals():
    SEED = 42
    rng = np.random.default_rng(SEED)
r = rng.random

rates = {
    "categoria_nan":        0.15,  # 15% Categoria ausente
    "produto_nan":          0.07,  # 7%  Produto ausente
    "preco_nan":            0.12,  # 12% Preco ausente
    "quantidade_nan":       0.08,  # 8%  Quantidade ausente
    "quantidade_str":       0.12,  # 12% Quantidade vira string
    "quantidade_zero_neg":  0.04,  # 4%  Quantidade vira 0/-1 (inválido)
    "data_nan":             0.03,  # 5%  Data ausente
    "data_invalida":        0.02,  # 4%  Data inválida (strings que quebram o parser)
    "dup_linha":            0.05,  # 5%  duplicatas idênticas
    "dup_chave":            0.05,  # 5%  duplicatas por (ID,Data) com valores diferentes
}

m = r(n) < rates["categoria_nan"];  df.loc[m, "Categoria"] = None
m = r(n) < rates["produto_nan"];    df.loc[m, "Produto"] = None
m = r(n) < rates["preco_nan"];      df.loc[m, "Preco"] = None
m = r(n) < rates["quantidade_nan"]; df.loc[m, "Quantidade"] = None
m = r(n) < rates["data_nan"];       df.loc[m, "Data"] = None   

m = r(n) < rates["quantidade_str"]
df.loc[m, "Quantidade"] = (
    pd.to_numeric(df.loc[m, "Quantidade"], errors= "coerce")
      .fillna(0).astype(int).astype(str)
)

m = r(n) < rates["data_invalida"];  df.loc[m, "Data"] = "2023-13-01"
m = r(n) < rates["data_invalida"];  df.loc[m, "Data"] = "31/02/2023"

k = max(1, int(rates["dup_chave"] * n))
idx = rng.choice(df.index, size=k, replace=False)
dups = df.loc[idx].copy()

dups["Preco"] = pd.to_numeric(dups["Preco"], errors="coerce") * (1 + rng.normal(0, 0.05, size=len(dups)))
dups["Quantidade"] = (
    pd.to_numeric(dups["Quantidade"], errors="coerce").fillna(1).astype(int)
    + rng.integers(-1, 2, size=len(dups))
)

df = pd.concat([df, dups], ignore_index=True)
df = df.sort_values(by=['ID'], kind='mergesort').reset_index(drop=True)
df.to_csv(raw_path, index=False, encoding="utf-8")
print(f"[OK] Sujeiras aplicadas e salvo: {raw_path} | linhas={len(df)}")

[OK] Sujeiras aplicadas e salvo: C:\DEV\Teste_Analytics_CristovamPaulo\data\raw\vendas_2023.csv | linhas=315


C:\Users\crish\AppData\Local\Temp\ipykernel_3656\3504816063.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['0' '5' '3' '4' '2' '4' '4' '6' '3' '1' '5' '0' '4' '2' '5' '3' '9' '3'
 '4' '2' '2' '6' '10']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[m, "Quantidade"] = (


In [26]:
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display

def diagnosticar_csv_plus(
    path_csv,
    keys_opcionais=None,                 # ex.: [("ID","Data"), ("Produto","Data")]
    periodo=("2023-01-01","2024-06-30"), # janela válida para Data
    mostrar_head=3,
    sample_k=3                           # quantos exemplos de linhas problemáticas mostrar
):
    df = pd.read_csv(path_csv)
    nlin, ncol = df.shape
    print(f"Arquivo: {path_csv} | shape = ({nlin}, {ncol})\n")

    # 1) Head e dtypes
    if mostrar_head:
        print(f"Amostra (head {mostrar_head}):")
        display(df.head(mostrar_head))
    print("\nTipos por coluna:")
    display(df.dtypes.to_frame("dtype"))

    # 2) Nulos (contagem e %)
    print("\nValores ausentes por coluna:")
    nulos = df.isna().sum()
    pct   = (nulos / max(len(df),1) * 100).round(2)
    out_nulos = (pd.DataFrame({"nulos": nulos, "pct_%": pct})
                   .sort_values("nulos", ascending=False))
    display(out_nulos)

    # 3) Duplicatas (linha inteira + por chaves)
    print("\nDuplicatas (linha inteira):", int(df.duplicated().sum()))
    if keys_opcionais:
        rows = []
        # normaliza: aceitar ("ID","Data") ou [("ID","Data"), ...]
        if isinstance(keys_opcionais, (tuple, list)) and all(isinstance(x, str) for x in keys_opcionais):
            keys_opcionais = [tuple(keys_opcionais)]
        for ch in keys_opcionais:
            ch = tuple(ch)
            rows.append({"chaves": str(ch),
                         "duplicatas": int(df.duplicated(subset=list(ch)).sum())})
        print("\nDuplicatas por chave:")
        display(pd.DataFrame(rows).sort_values("duplicatas", ascending=False, ignore_index=True))

    # 4) Texto que parece número (tipo misto)
    print("\nColunas 'object' que parecem numéricas:")
    rows = []
    for col in df.columns:
        if df[col].dtype == "object":
            conv = pd.to_numeric(df[col], errors="coerce")
            mask = df[col].notna() & conv.notna()
            qtd  = int(mask.sum())
            if qtd > 0:
                exemplos = (df.loc[mask, col].astype(str).drop_duplicates()[:sample_k].tolist())
                rows.append({"coluna": col, "qtd_texto_numerico": qtd, "pct_%": round(qtd/len(df)*100,2),
                             "exemplos": exemplos})
    if rows:
        display(pd.DataFrame(rows).sort_values("qtd_texto_numerico", ascending=False, ignore_index=True))
    else:
        print("  (nenhuma)")

    # 5) Problemas específicos por coluna

    # 5.1 Data: inválida e fora do período
    if "Data" in df.columns:
        dt = pd.to_datetime(df["Data"], errors="coerce")
        mask_invalida = dt.isna() & df["Data"].notna()     # strings inválidas (não conta NaN já acima)
        inicio, fim = pd.to_datetime(periodo[0]), pd.to_datetime(periodo[1])
        mask_fora = dt.notna() & ((dt < inicio) | (dt > fim))
        print("\n[Data] inválida:", int(mask_invalida.sum()), " | fora do período:", int(mask_fora.sum()))
        if mask_invalida.any():
            print("Exemplos Data inválida:")
            display(df.loc[mask_invalida].head(sample_k))
        if mask_fora.any():
            print("Exemplos Data fora do período:")
            display(df.loc[mask_fora].head(sample_k))

    # 5.2 Quantidade: não numérica, zeros/negativos
    if "Quantidade" in df.columns:
        q_conv = pd.to_numeric(df["Quantidade"], errors="coerce")
        mask_nao_num = df["Quantidade"].notna() & q_conv.isna()
        mask_zero_neg = q_conv.notna() & (q_conv <= 0)
        print("\n[Quantidade] não numérica:", int(mask_nao_num.sum()), " | <= 0:", int(mask_zero_neg.sum()))
        if mask_nao_num.any():
            print("Exemplos Quantidade não numérica:")
            display(df.loc[mask_nao_num].head(sample_k))
        if mask_zero_neg.any():
            print("Exemplos Quantidade <= 0:")
            display(df.loc[mask_zero_neg].head(sample_k))

    # 5.3 Preco: não numérico, <= 0
    if "Preco" in df.columns:
        p_conv = pd.to_numeric(df["Preco"], errors="coerce")
        mask_nao_num = df["Preco"].notna() & p_conv.isna()
        mask_le_zero = p_conv.notna() & (p_conv <= 0)
        print("\n[Preco] não numérico:", int(mask_nao_num.sum()), " | <= 0:", int(mask_le_zero.sum()))
        if mask_nao_num.any():
            print("Exemplos Preco não numérico:")
            display(df.loc[mask_nao_num].head(sample_k))
        if mask_le_zero.any():
            print("Exemplos Preco <= 0:")
            display(df.loc[mask_le_zero].head(sample_k))

    return df  # permite encadear outras análises

resultado = diagnosticar_csv_plus(raw_path, keys_opcionais=[("ID","Data")])
resultado.head()



Arquivo: C:\DEV\Teste_Analytics_CristovamPaulo\data\raw\vendas_2023.csv | shape = (315, 6)

Amostra (head 3):


,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-08-13,Mouse Óptico,Eletrônicos,1.0,275.60
1,2,2023-10-12,Boné Snapback,Moda,4.0,139.26
2,3,2023-09-27,Camiseta Básica,Moda,NaN,68.88



Tipos por coluna:


,dtype
ID,int64
Data,object
Produto,object
Categoria,object
Quantidade,float64
Preco,float64



Valores ausentes por coluna:


...

,ID,Data,Produto,Categoria,Quantidade,Preco
14,15,31/02/2023,Calça Jeans,Moda,3.0,96.59
36,37,2023-13-01,Jogo de Tabuleiro,NaN,7.0,157.34
113,111,2023-13-01,Liquidificador Turbo,Casa & Cozinha,5.0,113.17



[Quantidade] não numérica: 0  | <= 0: 2
Exemplos Quantidade <= 0:


,ID,Data,Produto,Categoria,Quantidade,Preco
16,17,2023-04-02,Mochila Fitness,Esporte,0.0,135.09
175,169,2023-02-21,Liquidificador Turbo,Casa & Cozinha,0.0,NaN



[Preco] não numérico: 0  | <= 0: 0


,ID,Data,Produto,Categoria,Quantidade,Preco
0,1,2023-08-13,Mouse Óptico,Eletrônicos,1.0,275.60
1,2,2023-10-12,Boné Snapback,Moda,4.0,139.26
2,3,2023-09-27,Camiseta Básica,Moda,NaN,68.88
3,4,2023-02-19,Camiseta Dry,Esporte,4.0,54.67
4,5,NaN,Garrafa Térmica,NaN,5.0,60.13


In [ ]:
from pathlib import Path
from typing import Dict, Tuple
import numpy as np
import pandas as pd


COLS = ["ID", "Data", "Produto", "Categoria", "Quantidade", "Preco"]

_base_root = globals().get("ROOT", Path.cwd().resolve())
if isinstance(_base_root, Path) and _base_root.name == "Parte01":
    _base_root = _base_root.parent

_data_dir = globals().get("DATA_DIR", _base_root / "data")
_raw_dir = globals().get("RAW_DIR", _data_dir / "raw")
_processed_dir = globals().get("PROCESSED_DIR", _data_dir / "processed")

_raw_dir.mkdir(parents=True, exist_ok=True)
_processed_dir.mkdir(parents=True, exist_ok=True)

RAW_PATH = (_raw_dir / "vendas_2023.csv").resolve()
CLEAN_PATH = (_processed_dir / "data_clean.csv").resolve()

print("RAW_PATH  :", RAW_PATH)
print("CLEAN_PATH:", CLEAN_PATH)

# ---------------------------
# Helpers
# ---------------------------
def _first_non_null(s: pd.Series):
    s2 = s.dropna()
    return s2.iloc[0] if not s2.empty else pd.NA

def _mode_or_first(s: pd.Series):
    s2 = s.dropna()
    if s2.empty:
        return pd.NA
    mode = s2.mode()
    return mode.iloc[0] if not mode.empty else s2.iloc[0]

def _to_num(s: pd.Series):
    return (
        pd.to_numeric(
            s.astype(str)
             .str.replace('.', '', regex=False)
             .str.replace(',', '.', regex=False),
            errors='coerce'
        )
    )

def _stringify(s: pd.Series):
    return (
        s.astype(str)
         .str.strip()
         .replace({'': pd.NA, 'nan': pd.NA})
    )

def _maybe_to_datetime(s: pd.Series):
    d = pd.to_datetime(s, errors='coerce', format='mixed')
    return d.dt.strftime('%Y-%m-%d')

def clean_sales_csv(
    input_path: Path,
    *,
    output_path: Path | None = None,
    key_cols=("ID", "Data"),
    price_decimals: int = 2,
    save_float_format: str = "%.2f",
) -> Tuple[pd.DataFrame, Dict[str, int]]:
    input_path = Path(input_path)
    if output_path is None:
        output_path = CLEAN_PATH
    else:
        output_path = Path(output_path)

    report: Dict[str, int] = {}

    if not input_path.exists():
        raise FileNotFoundError(f"Arquivo de entrada n�o encontrado: {input_path}")

    output_path.parent.mkdir(parents=True, exist_ok=True)

    df = pd.read_csv(input_path)
    df = df.copy()

    missing_cols = [c for c in COLS if c not in df.columns]
    if missing_cols:
        raise ValueError(f"Colunas esperadas ausentes: {missing_cols}")

    df = df[COLS].copy()
    report["linhas_iniciais"] = int(len(df))

    df["Data"] = _maybe_to_datetime(df["Data"])
    report["datas_invalidas"] = int(df["Data"].isna().sum())

    df["Produto"] = _stringify(df["Produto"])
    df["Categoria"] = _stringify(df["Categoria"])

    duplicates_exact = int(df.duplicated(keep='first').sum())
    report["duplicatas_exatas"] = duplicates_exact
    df = df.drop_duplicates(keep='first').reset_index(drop=True)

    dup_id = df.duplicated(subset=["ID"], keep='first')
    report["ids_duplicados"] = int(dup_id.sum())
    df = df.loc[~dup_id].copy()

    df["Produto"] = df.groupby("Categoria")["Produto"].transform(_mode_or_first)
    df["Categoria"] = df.groupby("Produto")["Categoria"].transform(_mode_or_first)

    df["Quantidade"] = _to_num(df["Quantidade"])
    df["Preco"] = _to_num(df["Preco"])

    q_nonpos = int((df["Quantidade"] <= 0).fillna(False).sum())
    p_nonpos = int((df["Preco"] <= 0).fillna(False).sum())
    df.loc[df["Quantidade"] <= 0, "Quantidade"] = np.nan
    df.loc[df["Preco"] <= 0, "Preco"] = np.nan
    report["quantidade_nao_positiva_para_NaN"] = q_nonpos
    report["preco_nao_positivo_para_NaN"] = p_nonpos

    med_preco_prod = df.groupby("Produto", dropna=False)["Preco"].median()
    med_qtd_prod = df.groupby("Produto", dropna=False)["Quantidade"].median()

    p_missing_before = int(df["Preco"].isna().sum())
    df["Preco"] = df["Preco"].fillna(df["Produto"].map(med_preco_prod))
    df["Preco"] = df["Preco"].fillna(df["Preco"].median())
    report["preco_imputado"] = p_missing_before - int(df["Preco"].isna().sum())

    q_missing_before = int(df["Quantidade"].isna().sum())
    df["Quantidade"] = df["Quantidade"].fillna(df["Produto"].map(med_qtd_prod))
    df["Quantidade"] = df["Quantidade"].fillna(1)
    report["quantidade_imputada"] = q_missing_before - int(df["Quantidade"].isna().sum())

    dup_key_count = int(df.duplicated(subset=list(key_cols)).sum())
    report["duplicatas_por_chave_encontradas"] = dup_key_count

    def _agg_quantidade(s: pd.Series):
        v = _to_num(s)
        return float(np.nanmedian(v)) if v.notna().any() else np.nan

    def _agg_preco(s: pd.Series):
        v = _to_num(s)
        return float(np.nanmedian(v)) if v.notna().any() else np.nan

    df = (
        df.groupby(list(key_cols), as_index=False)
          .agg({
              "Produto": _mode_or_first,
              "Categoria": _mode_or_first,
              "Quantidade": _agg_quantidade,
              "Preco": _agg_preco
          })
    )

    prod_missing_before = int(df['Produto'].isna().sum())
    cat_missing_before = int(df['Categoria'].isna().sum())

    prod_to_cat = (
        df[['Produto', 'Categoria']].dropna().drop_duplicates()
          .groupby('Produto')['Categoria']
          .agg(lambda s: s.mode().iat[0])
    )
    df['Categoria'] = df['Categoria'].fillna(df['Produto'].map(prod_to_cat))

    cat_to_prod = (
        df[['Categoria', 'Produto']].dropna().drop_duplicates()
          .groupby('Categoria')['Produto']
          .agg(lambda s: s.mode().iat[0])
    )
    df['Produto'] = df['Produto'].fillna(df['Categoria'].map(cat_to_prod))

    prod_missing_after = int(df['Produto'].isna().sum())
    cat_missing_after = int(df['Categoria'].isna().sum())
    report['produto_imputado'] = prod_missing_before - prod_missing_after
    report['categoria_imputada'] = cat_missing_before - cat_missing_after

    mask_drop = df['Produto'].isna() | df['Categoria'].isna()
    if mask_drop.any():
        report['linhas_descartadas_sem_produto_ou_categoria'] = int(mask_drop.sum())
        df = df.loc[~mask_drop].copy()

    df["ID"] = df["ID"].astype("int64")
    df["Quantidade"] = pd.Series(df["Quantidade"]).round().astype("Int64")
    df["Preco"] = pd.Series(df["Preco"]).astype("float64").round(price_decimals)

    df["Produto"] = df["Produto"].astype("string")
    df["Categoria"] = df["Categoria"].astype("string")

    df = df.sort_values(by=list(key_cols), kind='mergesort').reset_index(drop=True)
    df.to_csv(output_path, index=False, encoding='utf-8', float_format=save_float_format)

    report["linhas_finais"] = int(len(df))
    report["arquivo_limpo"] = str(output_path)

    return df, report

if __name__ == "__main__":
    df_clean, resumo = clean_sales_csv(RAW_PATH, output_path=CLEAN_PATH)

    df_disk = pd.read_csv(CLEAN_PATH)
    print("len(df_disk) (lido do disco):", len(df_disk))

    print("Resumo da limpeza:")
    for k, v in resumo.items():
        print(f"- {k}: {v}")

    print("Dtypes:")
    print(df_clean.dtypes)

    print("Amostra do Preco (2 casas):")
    print(df_clean["Preco"].head(10))

    total_preview = (df_clean["Quantidade"].fillna(0).astype(float) * df_clean["Preco"]).round(2)
    print("Amostra do Total (2 casas):")
    print(total_preview.head(10))


RAW_PATH  : C:\DEV\Teste_Analytics_CristovamPaulo\data\raw\vendas_2023.csv
CLEAN_PATH: C:\DEV\Teste_Analytics_CristovamPaulo\data\processed\data_clean.csv
len(df_disk) (lido do disco): 234
Resumo da limpeza:
- linhas_iniciais: 315
- datas_invalidas: 17
- duplicatas_exatas: 0
- ids_duplicados: 15
- quantidade_nao_positiva_para_NaN: 2
- preco_nao_positivo_para_NaN: 0
- preco_imputado: 42
- quantidade_imputada: 16
- duplicatas_por_chave_encontradas: 0
- produto_imputado: 0
- categoria_imputada: 0
- linhas_descartadas_sem_produto_ou_categoria: 49
- linhas_finais: 234
- arquivo_limpo: C:\DEV\Teste_Analytics_CristovamPaulo\data\processed\data_clean.csv
Dtypes:
ID                     int64
Data                  object
Produto       string[python]
Categoria     string[python]
Quantidade             Int64
Preco                float64
dtype: object
Amostra do Preco (2 casas):
0      27560.0
1     139260.0
2      68880.0
3      54670.0
4      58140.0
5     141790.0
6      70320.0
7    1247060.0
8

In [ ]:
_data_dir = globals().get("DATA_DIR", _base_root / "data")
_processed_dir = globals().get("PROCESSED_DIR", _data_dir / "processed")

OUT_PRODUTOS = (_processed_dir / "vendas_por_produto.csv").resolve()

def analisar_vendas(clean_path=CLEAN_PATH, out_path=OUT_PRODUTOS):
    # 1) Carrega
    df = pd.read_csv(clean_path)

    # 2) Total com 2 casas (evita ruído desde já)
    df["Total"] = (df["Quantidade"] * df["Preco"]).round(2)

    # 3) Soma por produto
    vendas_produto = (
        df.groupby("Produto", as_index=False)["Total"]
          .sum()
    )

    # 4) Arredonda DE NOVO após o sum (proteção extra)
    vendas_produto["Total"] = vendas_produto["Total"].round(2)

    # 5) Ordena desc
    vendas_produto = vendas_produto.sort_values("Total", ascending=False).reset_index(drop=True)

    # 6) Salva com 2 casas no CSV
    out_path.parent.mkdir(parents=True, exist_ok=True)
    vendas_produto.to_csv(out_path, index=False, encoding="utf-8", float_format="%.2f")

    # 7) Print bonito
    print("\n=== Vendas por Produto (Top 5) ===")
    for _, r in vendas_produto.head(5).iterrows():
        valor = f"{r['Total']:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
        print(f"{r['Produto']}: R$ {valor}")

    # 8) Campeão
    top = vendas_produto.iloc[0]
    top_valor = f"{top['Total']:,.2f}".replace(",", "X").replace(".", ",").replace("X", ".")
    print(f"\nProduto campeão: {top['Produto']} com total de R$ {top_valor}")

    return vendas_produto

# Execute a análise agora
vendas_por_produto = analisar_vendas()



=== Vendas por Produto (Top 5) ===
Caixa Bluetooth: R$ 5.726.477.000,00
Bola Oficial: R$ 2.963.465.000,00
Liquidificador Turbo: R$ 1.975.118.000,00
Jaqueta Leve: R$ 1.706.437.000,00
Jogo de Tabuleiro: R$ 1.274.908.000,00

Produto campeão: Caixa Bluetooth com total de R$ 5.726.477.000,00
